In [1]:
# colab에서 드라이브 내 폴더 사용 위해 마운트
# from google.colab import drive

# drive.mount('/content/drive/')

In [2]:
# 폴더까지의 경로
# cd '/content/drive/MyDrive/GSA_Creative_Resarch/rock_sci_paper'

In [3]:
# 폴더 이동 확인
# ! ls

In [4]:
#필요한 라이브러리들 import
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import dataset
import model
import torch.nn.functional as F

c:\Users\USER\anaconda3\envs\fskd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# 각종 path및 하이퍼 파라미터 설정
data_path = 'C:\\Users\\USER\\Desktop\\GSH_CRP\\codes\\rock_sci_paper\\data\\ro_sci_pa_heo'
save_path = 'C:\\Users\\USER\\Desktop\\GSH_CRP\\codes\\rock_sci_paper\\model_para'
epochs = 100
batch_size = 16
learning_rate = 0.01
seed = 0
mode = 'lr'

In [6]:
# seed 설정
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
# transform 설정
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine((-45, 45), translate=(0.2,0.2)),
    transforms.Resize((256,256)),
    transforms.ColorJitter(brightness=0.5),
    transforms.ColorJitter(contrast=0.5),
    transforms.ColorJitter(saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

In [8]:
# dataset 설정
train_dataset = dataset.RockScissorsPaper(
    transform=train_transform,
    path = data_path,
    mode = 'train'
)
val_dataset = dataset.RockScissorsPaper(
    transform=test_transform,
    path = data_path,
    mode = 'val'
)
test_dataset = dataset.RockScissorsPaper(
    transform=test_transform,
    path = data_path,
    mode = 'test'
)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

240
30
30


In [9]:
# dataloader 설정
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
# 모델, 손실함수, 옵티마이저 설정
model = model.ResNet18(num_classes=3)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
def train(epoch):
    print('\nEpoch: %d'%epoch)
    # model train mode로 전환
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        if mode=='lr':
            h,w = inputs.shape[-2], inputs.shape[-1]
            lr_inputs = F.interpolate(inputs, (h//8, w//8))
            lr_inputs = F.interpolate(lr_inputs, (h,w))
            outputs, _, _, _, _ = model(lr_inputs)
        else:
            outputs, _, _, _, _ = model(inputs)
            
        _, pred = torch.max(outputs, 1)
        total += outputs.size(0)
        running_acc += (pred == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    total_loss = running_loss / len(trainloader)
    total_acc = 100 * running_acc / total
    print(f'Train epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

In [12]:
def test(epoch, loader, mode='val', mode2=False):
    print('\nEpoch: %d'%epoch)
    # model eval mode로 전환
    model.eval()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    label_dict = {0:0, 1:0, 2:0}
    correct_dict = {0:0, 1:0, 2:0}
    global BEST_SCORE
    with torch.no_grad():
        for (inputs, labels) in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            if mode=='lr':
                h,w = inputs.shape[-2], inputs.shape[-1]
                lr_inputs = F.interpolate(inputs, (h//8, w//8))
                lr_inputs = F.interpolate(lr_inputs, (h,w))
                outputs, _, _, _, _ = model(lr_inputs)
            else:
                outputs, _, _, _, _ = model(inputs)

            _, pred = torch.max(outputs, 1)
            
            total += outputs.size(0)
            running_acc += (pred == labels).sum().item()
            
            if mode2:
                for i in range(len(labels)):
                    label = labels[i]
                    label_dict[label.item()] += 1
                    if (pred==labels)[i]:
                        correct_dict[label.item()] += 1
            
            loss = criterion(outputs, labels)
            running_loss += loss.item()
        total_loss = running_loss / len(loader)
        total_acc = 100 * running_acc / total
        if mode2:
            print(label_dict)
            print(correct_dict)
        if total_acc >= BEST_SCORE and not mode=='test':
            path = os.path.join(save_path, f'lr_teacher.pth')
            torch.save(model.state_dict(), path)
            BEST_SCORE = total_acc
        print(f'Test epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

In [13]:
# 모델 학습 및 평가
BEST_SCORE = 0
for epoch in range(epochs):
    train(epoch)
    test(epoch, valloader)
    print(BEST_SCORE)


Epoch: 0
Train epoch : 0 loss : 1.231787101427714 Acc : 36.25%

Epoch: 0
Test epoch : 0 loss : 1.1886222958564758 Acc : 30.0%
30.0

Epoch: 1
Train epoch : 1 loss : 1.2473631024360656 Acc : 32.5%

Epoch: 1
Test epoch : 1 loss : 1.1490331888198853 Acc : 30.0%
30.0

Epoch: 2
Train epoch : 2 loss : 1.2114893515904746 Acc : 25.416666666666668%

Epoch: 2
Test epoch : 2 loss : 1.2050803899765015 Acc : 23.333333333333332%
30.0

Epoch: 3
Train epoch : 3 loss : 1.1644933899243672 Acc : 35.833333333333336%

Epoch: 3
Test epoch : 3 loss : 1.0993612408638 Acc : 43.333333333333336%
43.333333333333336

Epoch: 4
Train epoch : 4 loss : 1.1653952201207478 Acc : 34.583333333333336%

Epoch: 4
Test epoch : 4 loss : 1.1464014053344727 Acc : 23.333333333333332%
43.333333333333336

Epoch: 5
Train epoch : 5 loss : 1.1283426841100057 Acc : 35.416666666666664%

Epoch: 5
Test epoch : 5 loss : 1.48000830411911 Acc : 30.0%
43.333333333333336

Epoch: 6
Train epoch : 6 loss : 1.1504948059717814 Acc : 31.666666666666

In [ ]:
# 테스트셋에서 평가
model.load_state_dict(torch.load(os.path.join(save_path, f'lr_teacher.pth')))
test(-1, testloader, 'test', True)